# BART LARGE CNN

In [ ]:
import os
import sys
import json

import torch

from transformers import BartTokenizer, BartModel, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import logging as T_LOGGER
T_LOGGER.set_verbosity_error()

In [ ]:
checkpoint = "facebook/bart-large-cnn"

# tokenizer = BartTokenizer.from_pretrained(checkpoint)
# model = BartModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
long_str = """In the following example, you limit the notification delivery protocol to HTTPS.
You need to know how to write your own policy for the topic because the Amazon 
SNS AddPermission action doesn't let you specify a protocol restriction when granting someone 
access to your topic. In this case, you would write your own policy, 
and then use the SetTopicAttributes action to set the topic's Policy attribute to your new policy."""

In [ ]:
input_ids = tokenizer.encode(long_str, truncation=True, padding=True, return_tensors='pt')
input_ids

In [ ]:
tokenizer.decode(input_ids[0], skip_special_tokens=True)

In [ ]:
outputs = model.generate(
    input_ids,
    max_length=128,
    do_sample=True,
    top_k=35,
    top_p=0.9,
    early_stopping=True,
    num_return_sequences=1
    )

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
model = model.to(memory_format=torch.channels_last)
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [ ]:
outputs = model.generate(
    input_ids,
    max_length=128,
    do_sample=True,
    top_k=35,
    top_p=0.9,
    early_stopping=True,
    num_return_sequences=1
    )

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)